In [ ]:
import dspy

openai = dspy.LM("openai/gpt-4.1-nano", cache=False)
dspy.configure(lm=openai)

In [1]:
from phoenix.otel import register

register(
	project_name="dspy-demo",
	endpoint="http://localhost:6006/v1/traces",
	auto_instrument=True,
	verbose=False
)

# Signatures

Signatures in DSPy serve the same purpose as a function signature in Python. They define the inputs and outputs of a DSPy module, allowing DSPy to understand what to expect and what to produce. Signatures are used to validate the inputs and outputs of a module, ensuring that they conform to the expected types and formats.

In [ ]:
from typing import Optional

simple_signature = dspy.Signature("question -> answer")
typed_signature = dspy.Signature("question: str -> answer: str")


In [12]:
predictor = dspy.Predict(simple_signature)

predictor(question="When did Iowa become a state?")

Prediction(
    answer='Iowa became a state on December 28, 1846.'
)

In [15]:
typed_predictor = dspy.Predict(typed_signature)

typed_predictor(question="When were fireworks invented?")

Prediction(
    answer='Fireworks were invented in China around the 7th century during the Tang Dynasty, with their use becoming more widespread in the following centuries.'
)

In [41]:
class ClassSignature(dspy.Signature):
	"""
	Answers a question based on the provided input.
	"""

	question: str = dspy.InputField()
	contexts: Optional[list[str]] = dspy.InputField(desc="List of contexts to answer the question from.")
	answer: str = dspy.OutputField(desc="Answer to the question, which is succinct and provides no additional information or preamble.")

class_predictor = dspy.Predict(ClassSignature)

class_predictor(question="What is DSPy?", contexts=None)

Prediction(
    answer='DSPy is a Python library designed for digital signal processing tasks, providing tools for analyzing, filtering, and manipulating signals efficiently.'
)

In [ ]:
class_predictor(question="What is DSPy?", contexts=[
    """DSPy is a declarative framework for building modular AI software. It allows you to iterate fast on structured code, rather than brittle strings, and offers algorithms that compile AI programs into effective prompts and weights for your language models, whether you're building simple classifiers, sophisticated RAG pipelines, or Agent loops.

	Instead of wrangling prompts or training jobs, DSPy (Declarative Self-improving Python) enables you to build AI software from natural-language modules and to generically compose them with different models, inference strategies, or learning algorithms. This makes AI software more reliable, maintainable, and portable across models and strategies."""
])

Prediction(
    answer='DSPy is a declarative framework for building modular AI software that allows for fast iteration on structured code and compiles AI programs into effective prompts and weights for language models.'
)

# Modules

Modules are used in DSPy to define structure to an AI program. They are the building blocks of DSPy programming and there are several built-in modules that can be leveraged.

* dspy.Predict: Standard zero-shot prediction module.
* dspy.ChainOfThought: Chain of thought module that allows for reasoning over multiple steps.
* dspy.ReAct: ReAct module that allows for reasoning and action-taking (tool calling).
* dspy.ProgramOfThought: A DSPy module that runs Python programs to solve a problem.
* More modules can be found in the [DSPy documentation](https://dspy.ai/api/modules/Module/).

In [40]:
chain_of_thought = dspy.ChainOfThought(typed_signature)

print("Basic prediction:", predictor(question="How many r's are in the word strawberry?").answer)

chain_of_thought(question="How many r's are in the word strawberry?")

Basic prediction: There are 2 r's in the word strawberry.


Prediction(
    reasoning='The word "strawberry" consists of the letters s, t, r, a, w, b, e, r, r, y. Counting the number of \'r\'s in this word, we find there are three: one in the third position, and two more towards the end.',
    answer="There are 3 r's in the word strawberry."
)

In [ ]:
from typing import Literal

@dspy.Tool
def do_math(x: float, y: float, operation: Literal['+', '-', '*', '/', '^']) -> float:
	"""
	Performs a mathematical operation on two numbers.
	"""
	if operation == '+':
		return x + y
	elif operation == '-':
		return x - y
	elif operation == '*':
		return x * y
	elif operation == '/':
		return x / y
	elif operation == '^':
		return x ** y
	else:
		raise ValueError("Invalid operation. Supported operations are: '+', '-', '*', '/', '^'.")

math_signature = dspy.Signature("question -> answer: float")

math_agent = dspy.ReAct(
	math_signature,
	tools=[do_math],
)

In [34]:
from pprint import pprint
math_response = math_agent(question="Raise the largest prime number less than 100 to the power of 2.")

print("Trajectory")
pprint(math_response.trajectory)
print("reasoning:", math_response.reasoning)
print("answer:", math_response.answer)

Trajectory
{'observation_0': 9409.0,
 'observation_1': 'Completed.',
 'thought_0': 'The largest prime number less than 100 is 97. I need to '
              'calculate 97 raised to the power of 2.',
 'thought_1': 'The calculation of 97 raised to the power of 2 has been '
              'completed, resulting in 9409. Therefore, I can now conclude the '
              'task and produce the answer.',
 'tool_args_0': {'operation': '^', 'x': 97, 'y': 2},
 'tool_args_1': {},
 'tool_name_0': 'do_math',
 'tool_name_1': 'finish'}
reasoning: The largest prime number less than 100 is 97. Raising 97 to the power of 2 involves calculating 97 squared, which equals 9409. This calculation has been confirmed in the previous step, so the answer is 9409.0.
answer: 9409.0
